Notes:
- new data from remm se output
- old data from wfrc data portal
- before running I standarized the column names
- Filter city areas

Create a Feature Layer for each combination of Map Display Geography (4) x Variables (7) wherein the attribute fields are 

  Like this for Households (for the other 6 variables substitute hh_ with: pop_, ijb_, ojb, rjb_, tjb_,  hji_)

TAZID, city, county, med_dist, small_dist, dev_acres, hh_19, hh_20, hh_21…hh_50…old_hh_20, old_hh_25, old_hh_50
Sml_DistID, county, dev_acres, hh_19, hh_20, hh_21…hh_50…old_hh_20, old_hh_25, old_hh_50
Med_DistID, county, dev_acres, hh_19, hh_20, hh_21…hh_50…old_hh_20, old_hh_25, old_hh_50
CityID, county, dev_acres, hh_19, hh_20, hh_21…hh_50…old_hh_20, old_hh_25, old_hh_50

# How the metrics are calculated:

tdm_output['ALLEMP'] = tdm_output['RETL']+tdm_output['FOOD']+tdm_output['MANU']+tdm_output['WSLE']+tdm_output['OFFI']+tdm_output['GVED']+tdm_output['HLTH']+tdm_output['OTHR']+tdm_output['FM_AGRI']+tdm_output['FM_MING']+tdm_output['FM_CONS']+tdm_output['HBJ']
    
    tdm_output['RETEMP'] = tdm_output['RETL'] + tdm_output['FOOD']
    tdm_output['INDEMP'] = tdm_output['MANU'] + tdm_output['WSLE']
    tdm_output['OTHEMP'] = tdm_output['OFFI'] + tdm_output['GVED']+ tdm_output['HLTH'] + tdm_output['OTHR']
    tdm_output['TOTEMP'] = tdm_output['RETEMP'] + tdm_output['INDEMP']+ tdm_output['OTHEMP']


In [1]:
import arcpy
from arcpy import env
import os
import glob
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd
import zipfile
from datetime import date
import shutil

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [2]:
# import datasets
# parcels = r'.\\Inputs\\remm_base_year_20220513.gdb\\parcels_2019'
taz832 = r".\Inputs\TAZ_832_ID_Only.shp"
# taz900 = r".\Inputs\TAZ_900.shp"
taz900 = r".\Inputs\TAZ_900.gdb\TAZ"
city_areas = r".\Inputs\TAZ_900.gdb\CITYAREA"
med_districts = r".\Inputs\TAZ_900.gdb\DISTMED"
counties = r".\Inputs\TAZ_900.gdb\COUNTY"
# small_districts = ???
# regions_df = pd.DataFrame.spatial.from_featureclass(regions_shp)

taz832_sdf = pd.DataFrame.spatial.from_featureclass(taz832)
taz900_sdf = pd.DataFrame.spatial.from_featureclass(taz900)
city_areas_sdf = pd.DataFrame.spatial.from_featureclass(city_areas)[['CityArea','SHAPE']].copy()
med_districts_sdf = pd.DataFrame.spatial.from_featureclass(med_districts)[['DISTMED','SHAPE']].copy()
counties_sdf = pd.DataFrame.spatial.from_featureclass(counties)[['CO_NAME','SHAPE']].copy()

In [3]:
# store the paths to the new SE outputs
pm_folder = r".\New_SE_Data"
new_taz_se = glob.glob(os.path.join(pm_folder,'SE_WF_*.csv'))
len(new_taz_se)

32

In [4]:
# store the paths to the old SE outputs
old_se_folder = r".\Old_SE_Data"
old_taz_se = glob.glob(os.path.join(old_se_folder,'*_TAZ.csv'))
old_city_se = glob.glob(os.path.join(old_se_folder,'*_City_Area.csv'))

In [5]:
outputs = r'.\Outputs'
if not os.path.exists(outputs):
    os.makedirs(outputs)

scratch = os.path.join(outputs, 'scratch.gdb')
if not arcpy.Exists(scratch):
    arcpy.CreateFileGDB_management(outputs, 'scratch.gdb')


In [6]:
# year = os.path.splitext(os.path.basename(new_taz_se[0]))[0].split('_')[-1]
# x = pd.read_csv(new_taz_se[0])
# x['OFFI'] = x['OFFI'] + x['GVED'] + x['HLTH']
# x['TPCL'] = x['OFFI'] + x['RETEMP'] + x['INDEMP']
# x['HJI'] = x['TOTHH']*1.8 + x['TOTEMP']
# x = x[[';TAZID', 'TOTHH', 'HHPOP', 'TOTEMP','RETEMP', 'INDEMP', 'OFFI', 'TPCL', 'HJI']].copy()
# x.columns = ['TAZID', f'HH_{year}', f'POP_{year}', f'EMP_{year}', f'RTL_{year}', f'IND_{year}', f'OFFI_{year}', f'TPCL_{year}', f'HJI_{year}']
# x


### Process New SE Data

In [7]:
base = taz900_sdf[['SA_TAZID', 'DISTMED', 'CityArea', 'CO_NAME','SHAPE']].copy()
base.shape

for csv in new_taz_se:

    year = os.path.splitext(os.path.basename(csv))[0].split('_')[-1]
    df = pd.read_csv(csv)
    df['TPCL'] = df['TOTEMP']
    df['HJI'] = df['TOTHH']*1.8 + df['TPCL']
    df = df[[';TAZID', 'TOTHH', 'HHPOP', 'ALLEMP','RETEMP', 'INDEMP', 'OTHEMP', 'TPCL', 'HJI']].copy()
    df.columns = ['SA_TAZID', f'HH_{year}', f'POP_{year}', f'AEMP_{year}', f'RTL_{year}', f'IND_{year}', f'OTHR_{year}', f'TPCL_{year}', f'HJI_{year}']
    
    base = base.merge(df, on='SA_TAZID', how='left')

new_se = base

In [8]:
new_se.shape

(3546, 261)

In [9]:
# export
new_se_taz_900 = os.path.join(scratch, '_01_new_se_taz_900')
new_se.spatial.to_featureclass(location=new_se_taz_900,sanitize_columns=False)

'e:\\Projects\\REMM-Process-SE-Output-For-Web\\Outputs\\scratch.gdb\\_01_new_se_taz_900'

### Process Old SE Data

In [10]:
se_dict =  {'TOTHH':'HH',    
            'HHPOP':'POP',
            'RETEMP':'RTL',
            'TOTEMP':'TPCL',
            'ALLEMP':'AEMP',
            'INDEMP':'IND',
            'OTHEMP':'OTHR'}

In [11]:
# test = pd.read_csv(old_taz_se[0])
# category = se_dict[test.iloc[0]['SECategory']]
# val_cols = [col for col in list(test.columns) if 'YEAR' in col and  'D' not in col and int(col[4:]) >= 2019]
# test = test[['CO_TAZID'] + val_cols]
# new_val_cols = [col.replace('YEAR',category + '_') for col in val_cols]
# test.columns = ['CO_TAZID'] + new_val_cols
# test

In [12]:
old_base = taz832_sdf[['CO_TAZID','SHAPE']].copy()
old_base.shape

for csv in old_taz_se:

    df = pd.read_csv(csv)
    category = se_dict[df.iloc[0]['SECategory']]
    val_cols = [col for col in list(df.columns) if 'YEAR' in col and  'D' not in col and int(col[4:]) >= 2019]
    df = df[['CO_TAZID'] + val_cols]
    new_val_cols = [col.replace('YEAR',category + '_') for col in val_cols]
    df.columns = ['CO_TAZID'] + new_val_cols
    old_base = old_base.merge(df, on='CO_TAZID',how='left')

old_se = old_base
old_se.rename({'CO_TAZID': 'COTAZID832'}, axis=1, inplace=True)

In [13]:
# calculate household jobs index
for year in range(2019,2051):
    old_se['HJI_{}'.format(year)] = (old_se['HH_{}'.format(year)] * 1.8) + old_se['TPCL_{}'.format(year)]

In [14]:
old_se.shape

(2881, 258)

In [15]:
# export
old_se_taz_832 = os.path.join(scratch, '_02_old_se_taz_832')
old_se.spatial.to_featureclass(location=old_se_taz_832,sanitize_columns=False)

'e:\\Projects\\REMM-Process-SE-Output-For-Web\\Outputs\\scratch.gdb\\_02_old_se_taz_832'

## Identity

In [16]:
# compare output dimensions
old_se.shape == new_se.shape

False

In [17]:
# merge geometry between taz 832 and output geometries (taz900, )
taz900_identity = arcpy.Identity_analysis(taz900, old_se_taz_832, os.path.join(scratch, '_03_taz832_taz900_identity'))
arcpy.management.DeleteField(taz900_identity,['SA_TAZID', 'DISTMED', 'CityArea', 'CO_NAME'], "KEEP_FIELDS")

<Result '.\\Outputs\\scratch.gdb\\_03_taz832_taz900_identity'>

## Apportion the attributes from old geometry to the new ones

1. The Apportion tool in arcpy is still lame 
2. Open the arcgis pro, ensure the identity layer are present in the map
3. Delete previous apportioning files if present
4. Paste the text from the file, "apportion_command_for_arcgis_pro.txt", in the arcgis pro python window to get the apportion output

In [18]:
os.startfile(r".\REMM-Process-Progression-Metrics-For-Web.aprx")
os.startfile(r".\apportion_command_for_arcgis_pro_v2.txt")

In [19]:
# apportion = arcpy.ApportionPolygon_analysis(old_se_taz_832, app_columns, identity, os.path.join(outputs, 'old_se_apportion_to_taz900.shp'), "AREA", "", "", "MAINTAIN_GEOMETRIES")
old_taz_apportion = os.path.join(scratch,'_05_old_se_apportion_to_taz900')

## Dissolve to summarize at geometry

In [22]:
app_columns = list(new_se.columns)[4:]
stat_fields = [[col,'SUM'] for col in app_columns]

#########
# New
#########

# taz (9.0.0)
new_se_taz900_dissolve = arcpy.Dissolve_management(new_se_taz_900, os.path.join(scratch, '_06_new_se_taz900_dissolve'),
                          'SA_TAZID', stat_fields, "MULTI_PART")

# distmed (9.0.0)
new_se_distmed_dissolve = arcpy.Dissolve_management(new_se_taz_900, os.path.join(scratch, '_06_new_se_distmed_dissolve'),
                          'DISTMED', stat_fields, "MULTI_PART")

# city area
new_se_cityarea_dissolve = arcpy.Dissolve_management(new_se_taz_900, os.path.join(scratch, '_06_new_se_cityarea_dissolve'),
                          'CityArea', stat_fields, "MULTI_PART")

# county
new_se_county_dissolve = arcpy.Dissolve_management(new_se_taz_900, os.path.join(scratch, '_06_new_se_county_dissolve'),
                          'CO_NAME', stat_fields, "MULTI_PART")



In [23]:
#########
# Old
#########

# taz (9.0.0)
old_se_taz900_dissolve = arcpy.Dissolve_management(old_taz_apportion, os.path.join(scratch, '_06_old_se_taz900_dissolve'),
                          'SA_TAZID', stat_fields, "MULTI_PART")

# distmed (9.0.0)
old_se_distmed_dissolve = arcpy.Dissolve_management(old_taz_apportion, os.path.join(scratch, '_06_old_se_distmed_dissolve'),
                          'DISTMED', stat_fields, "MULTI_PART")

# city area
old_se_cityarea_dissolve = arcpy.Dissolve_management(old_taz_apportion, os.path.join(scratch, '_06_old_se_cityarea_dissolve'),
                          'CityArea', stat_fields, "MULTI_PART")

# county
old_se_county_dissolve = arcpy.Dissolve_management(old_taz_apportion, os.path.join(scratch, '_06_old_se_county_dissolve'),
                          'CO_NAME', stat_fields, "MULTI_PART")


In [ ]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [ ]:
# read in new dataframes
new_se_taz900_dissolve_df = pd.DataFrame.spatial.from_featureclass(new_se_taz900_dissolve[0])
new_se_distmed_dissolve_df = pd.DataFrame.spatial.from_featureclass(new_se_distmed_dissolve[0])
new_se_cityarea_dissolve_df = pd.DataFrame.spatial.from_featureclass(new_se_cityarea_dissolve[0])
new_se_county_dissolve_df = pd.DataFrame.spatial.from_featureclass(new_se_county_dissolve[0])
old_se_taz900_dissolve_df = pd.DataFrame.spatial.from_featureclass(old_se_taz900_dissolve[0])
old_se_distmed_dissolve_df = pd.DataFrame.spatial.from_featureclass(old_se_distmed_dissolve[0])
old_se_cityarea_dissolve_df = pd.DataFrame.spatial.from_featureclass(old_se_cityarea_dissolve[0])
old_se_county_dissolve_df = pd.DataFrame.spatial.from_featureclass(old_se_county_dissolve[0])

# fill NAs
new_se_taz900_dissolve_df = fill_na_sedf(new_se_taz900_dissolve_df)
new_se_distmed_dissolve_df = fill_na_sedf(new_se_distmed_dissolve_df)
new_se_cityarea_dissolve_df = fill_na_sedf(new_se_cityarea_dissolve_df)
new_se_county_dissolve_df = fill_na_sedf(new_se_county_dissolve_df)
old_se_taz900_dissolve_df = fill_na_sedf(old_se_taz900_dissolve_df)
old_se_distmed_dissolve_df = fill_na_sedf(old_se_distmed_dissolve_df)
old_se_cityarea_dissolve_df = fill_na_sedf(old_se_cityarea_dissolve_df)
old_se_county_dissolve_df = fill_na_sedf(old_se_county_dissolve_df)

# round !! come up with something better
new_se_taz900_dissolve_df = new_se_taz900_dissolve_df.round()
new_se_distmed_dissolve_df = new_se_distmed_dissolve_df.round()
new_se_cityarea_dissolve_df = new_se_cityarea_dissolve_df.round()
new_se_county_dissolve_df = new_se_county_dissolve_df.round()
old_se_taz900_dissolve_df = old_se_taz900_dissolve_df.round()
old_se_distmed_dissolve_df = old_se_distmed_dissolve_df.round()
old_se_cityarea_dissolve_df = old_se_cityarea_dissolve_df.round()
old_se_county_dissolve_df = old_se_county_dissolve_df.round()

# rename SA_TAZID column
new_se_taz900_dissolve_df.rename({'SA_TAZID':'TAZID'}, axis=1, inplace=True)
old_se_taz900_dissolve_df.rename({'SA_TAZID':'TAZID'}, axis=1, inplace=True)

del new_se_distmed_dissolve_df['SHAPE']
del old_se_distmed_dissolve_df['SHAPE']
del new_se_cityarea_dissolve_df['SHAPE']
del old_se_cityarea_dissolve_df['SHAPE']
del new_se_county_dissolve_df['SHAPE']
del old_se_county_dissolve_df['SHAPE']

In [ ]:
# Remove 'SUM' from column names
to_replace = ['SUM_'+ col for col in app_columns]
replace_dict = dict(zip(to_replace, app_columns))

new_se_taz900_dissolve_df.rename(replace_dict, axis=1, inplace=True)
old_se_taz900_dissolve_df.rename(replace_dict, axis=1, inplace=True)

new_se_distmed_dissolve_df.rename(replace_dict, axis=1, inplace=True)
old_se_distmed_dissolve_df.rename(replace_dict, axis=1, inplace=True)

new_se_cityarea_dissolve_df.rename(replace_dict, axis=1, inplace=True)
old_se_cityarea_dissolve_df.rename(replace_dict, axis=1, inplace=True)

new_se_county_dissolve_df.rename(replace_dict, axis=1, inplace=True)
old_se_county_dissolve_df.rename(replace_dict, axis=1, inplace=True)

In [ ]:
# Create folders for deliverables

map_folder = os.path.join(outputs, "map")
if not os.path.exists(map_folder):
    os.makedirs(map_folder)

chart_folder = os.path.join(outputs, "chart")
if not os.path.exists(chart_folder):
    os.makedirs(chart_folder)

In [ ]:
#====================
# Process TAZ
#====================

# taz_folder = os.path.join(map_folder, "TAZ")
# if not os.path.exists(taz_folder):
#     os.makedirs(taz_folder)

taz_gdb = os.path.join(map_folder, 'TAZ.gdb')
if not arcpy.Exists(taz_gdb):
    arcpy.CreateFileGDB_management(map_folder, 'TAZ.gdb')


categories = ['HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI']
for c in categories:
    new_cols = [col for col in app_columns if col.split('_')[0] == c]
    new_temp_df = new_se_taz900_dissolve_df[['TAZID','SHAPE'] + new_cols].copy()
    old_temp_df = old_se_taz900_dissolve_df[['TAZID','SHAPE'] + new_cols].copy()
    

    new_new_names = [col.replace(c, 'N') for col in new_cols]
    new_old_names = [col.replace(c, 'O') for col in new_cols]
    new_rename_dict = dict(zip(new_cols, new_new_names))
    old_rename_dict = dict(zip(new_cols, new_old_names))
    new_temp_df.rename(new_rename_dict, axis=1, inplace=True)
    old_temp_df.rename(old_rename_dict, axis=1, inplace=True)

    del old_temp_df['SHAPE']
    merged = new_temp_df.merge(old_temp_df, on=['TAZID'], how='left')

    merged[new_new_names + new_old_names] = merged[new_new_names + new_old_names].fillna(value=0)

    # outfile = os.path.join(taz_folder, '{}_PROJECTIONS_by_TAZ.shp'.format(c))
    outfile = os.path.join(taz_gdb, '{}_PROJECTIONS_by_TAZ'.format(c))
    merged.spatial.to_featureclass(location=outfile, sanitize_columns=False)

    arcpy.AddField_management(outfile, "ACRES", "FLOAT")
    exp = "round(!SHAPE.AREA@ACRES!,2)"
    arcpy.CalculateField_management(outfile, "ACRES", exp, "PYTHON3")

In [ ]:
#====================
# Process DISTMED
#====================

distmed_gdb = os.path.join(map_folder, 'DISTMED.gdb')
if not arcpy.Exists(distmed_gdb):
    arcpy.CreateFileGDB_management(map_folder, 'DISTMED.gdb')

categories = ['HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI']
for c in categories:
    new_cols = [col for col in app_columns if col.split('_')[0] == c]
    new_temp_df = new_se_distmed_dissolve_df[['DISTMED'] + new_cols].copy()
    old_temp_df = old_se_distmed_dissolve_df[['DISTMED'] + new_cols].copy()
    

    new_new_names = [col.replace(c, 'N') for col in new_cols]
    new_old_names = [col.replace(c, 'O') for col in new_cols]
    new_rename_dict = dict(zip(new_cols, new_new_names))
    old_rename_dict = dict(zip(new_cols, new_old_names))
    new_temp_df.rename(new_rename_dict, axis=1, inplace=True)
    old_temp_df.rename(old_rename_dict, axis=1, inplace=True)

    merged = new_temp_df.merge(old_temp_df, on='DISTMED', how='left')
    merged = med_districts_sdf.merge(merged, on='DISTMED', how='left')

    merged[new_new_names + new_old_names] = merged[new_new_names + new_old_names].fillna(value=0)
    merged[new_new_names + new_old_names] = merged[new_new_names + new_old_names].round(0)

    # outfile = os.path.join(distmed_gdb, '{}_PROJECTIONS_by_DISTMED.shp'.format(c))
    outfile = os.path.join(distmed_gdb, '{}_PROJECTIONS_by_DISTMED'.format(c))
    merged.spatial.to_featureclass(location=outfile, sanitize_columns=False)

    arcpy.AddField_management(outfile, "ACRES", "FLOAT")
    exp = "round(!SHAPE.AREA@ACRES!,2)"
    arcpy.CalculateField_management(outfile, "ACRES", exp, "PYTHON3")


In [ ]:
#====================
# Process City Area
#====================

cityarea_gdb = os.path.join(map_folder, 'CITYAREA.gdb')
if not arcpy.Exists(cityarea_gdb):
    arcpy.CreateFileGDB_management(map_folder, 'CITYAREA.gdb')

categories = ['HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI']
for c in categories:
    new_cols = [col for col in app_columns if col.split('_')[0] == c]
    new_temp_df = new_se_cityarea_dissolve_df[['CityArea'] + new_cols].copy()
    old_temp_df = old_se_cityarea_dissolve_df[['CityArea'] + new_cols].copy()
    

    new_new_names = [col.replace(c, 'N') for col in new_cols]
    new_old_names = [col.replace(c, 'O') for col in new_cols]
    new_rename_dict = dict(zip(new_cols, new_new_names))
    old_rename_dict = dict(zip(new_cols, new_old_names))
    new_temp_df.rename(new_rename_dict, axis=1, inplace=True)
    old_temp_df.rename(old_rename_dict, axis=1, inplace=True)

    merged = new_temp_df.merge(old_temp_df, on='CityArea', how='left')
    merged = city_areas_sdf.merge(merged, on='CityArea', how='left')

    merged[new_new_names + new_old_names] = merged[new_new_names + new_old_names].fillna(value=0)

    # outfile = os.path.join(cityarea_folder, '{}_PROJECTIONS_by_CITYAREA.shp'.format(c))
    outfile = os.path.join(cityarea_gdb, '{}_PROJECTIONS_by_CITYAREA'.format(c))
    merged.spatial.to_featureclass(location=outfile, sanitize_columns=False)

    arcpy.AddField_management(outfile, "ACRES", "FLOAT")
    exp = "round(!SHAPE.AREA@ACRES!,2)"
    arcpy.CalculateField_management(outfile, "ACRES", exp, "PYTHON3")


In [ ]:
#====================
# Process County
#====================

county_gdb = os.path.join(map_folder, 'COUNTY.gdb')
if not arcpy.Exists(county_gdb):
    arcpy.CreateFileGDB_management(map_folder, 'COUNTY.gdb')

categories = ['HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI']
for c in categories:
    new_cols = [col for col in app_columns if col.split('_')[0] == c]
    new_temp_df = new_se_county_dissolve_df[['CO_NAME'] + new_cols].copy()
    old_temp_df = old_se_county_dissolve_df[['CO_NAME'] + new_cols].copy()
    

    new_new_names = [col.replace(c, 'N') for col in new_cols]
    new_old_names = [col.replace(c, 'O') for col in new_cols]
    new_rename_dict = dict(zip(new_cols, new_new_names))
    old_rename_dict = dict(zip(new_cols, new_old_names))
    new_temp_df.rename(new_rename_dict, axis=1, inplace=True)
    old_temp_df.rename(old_rename_dict, axis=1, inplace=True)

    merged = new_temp_df.merge(old_temp_df, on='CO_NAME', how='left')
    merged = counties_sdf.merge(merged, on='CO_NAME', how='left')

    merged[new_new_names + new_old_names] = merged[new_new_names + new_old_names].fillna(value=0)

    outfile = os.path.join(county_gdb, '{}_PROJECTIONS_by_COUNTY'.format(c))
    merged.spatial.to_featureclass(location=outfile, sanitize_columns=False)

    arcpy.AddField_management(outfile, "ACRES", "FLOAT")
    exp = "round(!SHAPE.AREA@ACRES!,2)"
    arcpy.CalculateField_management(outfile, "ACRES", exp, "PYTHON3")

In [ ]:
#=======================
# Process TAZ to json
#=======================

chart_taz_folder = os.path.join(chart_folder, 'TAZ')
if not os.path.exists(chart_taz_folder):
    os.makedirs(chart_taz_folder)
else:
    shutil.rmtree(chart_taz_folder)
    os.makedirs(chart_taz_folder)

new_df = new_se_taz900_dissolve_df.copy()
old_df = old_se_taz900_dissolve_df.copy()
id_col = 'TAZID'

categories = ('HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI')
cols = [col for col in new_df.columns if col.startswith(categories)]
new_new_cols = ['NEW_' + col for col in cols]
old_new_cols = ['OLD_' + col for col in cols]

new_rename_dict = dict(zip(cols, new_new_cols))
old_rename_dict = dict(zip(cols, old_new_cols))
new_df.rename(new_rename_dict, axis=1, inplace=True)
old_df.rename(old_rename_dict, axis=1, inplace=True)

del new_df['SHAPE']
del old_df['SHAPE']

df = new_df.merge(old_df, on=id_col, how='left')

for index, row in df.iterrows():
        f = open(os.path.join(chart_taz_folder, "TAZ_{}.json".format(int(row[id_col]))), "a")
        f.write("[\n")
        
        for cat in categories:  
            for year in range(2019,2051):
                
                new_col = '_'.join(['NEW', cat, str(year)])
                old_col = '_'.join(['OLD', cat, str(year)])

                f.write("\t{\n")
                f.write('''\t\t"C":"{}",\n'''.format(cat))
                f.write('''\t\t"Y":{},\n'''.format(year))
                
                f.write('''\t\t"N":{},\n'''.format(row[new_col]))
                f.write('''\t\t"O":{}\n'''.format(row[old_col]))
        
                if cat == "HJI" and year == 2050:
                    f.write("\t}\n")
                else:
                    f.write("\t},\n")

        f.write("]\n")
        f.close()

In [ ]:
#==========================
# Process DISTMED to json
#==========================

chart_DISTMED_folder = os.path.join(chart_folder, 'DISTMED')
if not os.path.exists(chart_DISTMED_folder):
    os.makedirs(chart_DISTMED_folder)
else:
    shutil.rmtree(chart_DISTMED_folder)
    os.makedirs(chart_DISTMED_folder)

new_df = new_se_distmed_dissolve_df.copy()
old_df = old_se_distmed_dissolve_df.copy()
id_col = 'DISTMED'

categories = ('HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI')
cols = [col for col in new_df.columns if col.startswith(categories)]
new_new_cols = ['NEW_' + col for col in cols]
old_new_cols = ['OLD_' + col for col in cols]

new_rename_dict = dict(zip(cols, new_new_cols))
old_rename_dict = dict(zip(cols, old_new_cols))
new_df.rename(new_rename_dict, axis=1, inplace=True)
old_df.rename(old_rename_dict, axis=1, inplace=True)

df = new_df.merge(old_df, on=id_col, how='left')

for index, row in df.iterrows():
        f = open(os.path.join(chart_DISTMED_folder, "{}_{}.json".format(id_col,int(row[id_col]))), "a")
        f.write("[\n")
        
        for cat in categories:  
            for year in range(2019,2051):
                
                new_col = '_'.join(['NEW', cat, str(year)])
                old_col = '_'.join(['OLD', cat, str(year)])

                f.write("\t{\n")
                f.write('''\t\t"C":"{}",\n'''.format(cat))
                f.write('''\t\t"Y":{},\n'''.format(year))
                
                f.write('''\t\t"N":{},\n'''.format(row[new_col]))
                f.write('''\t\t"O":{}\n'''.format(row[old_col]))
        
                if cat == "HJI" and year == 2050:
                    f.write("\t}\n")
                else:
                    f.write("\t},\n")

        f.write("]\n")
        f.close()

In [ ]:
#===========================
# Process CITYAREA to json
#===========================

chart_CITYAREA_folder = os.path.join(chart_folder, 'CITYAREA')
if not os.path.exists(chart_CITYAREA_folder):
    os.makedirs(chart_CITYAREA_folder)
else:
    shutil.rmtree(chart_CITYAREA_folder)
    os.makedirs(chart_CITYAREA_folder)


new_df = new_se_cityarea_dissolve_df.copy()
old_df = old_se_cityarea_dissolve_df.copy()
id_col = 'CityArea'

categories = ('HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI')
cols = [col for col in new_df.columns if col.startswith(categories)]
new_new_cols = ['NEW_' + col for col in cols]
old_new_cols = ['OLD_' + col for col in cols]

new_rename_dict = dict(zip(cols, new_new_cols))
old_rename_dict = dict(zip(cols, old_new_cols))
new_df.rename(new_rename_dict, axis=1, inplace=True)
old_df.rename(old_rename_dict, axis=1, inplace=True)

df = new_df.merge(old_df, on=id_col, how='left')

for index, row in df.iterrows():
        f = open(os.path.join(chart_CITYAREA_folder, "{}_{}.json".format(id_col,row[id_col])), "a")
        f.write("[\n")
        
        for cat in categories:  
            for year in range(2019,2051):
                
                new_col = '_'.join(['NEW', cat, str(year)])
                old_col = '_'.join(['OLD', cat, str(year)])

                f.write("\t{\n")
                f.write('''\t\t"C":"{}",\n'''.format(cat))
                f.write('''\t\t"Y":{},\n'''.format(year))
                
                f.write('''\t\t"N":{},\n'''.format(row[new_col]))
                f.write('''\t\t"O":{}\n'''.format(row[old_col]))
        
                if cat == "HJI" and year == 2050:
                    f.write("\t}\n")
                else:
                    f.write("\t},\n")

        f.write("]\n")
        f.close()

In [ ]:
#===========================
# Process COUNTY to json
#===========================

chart_COUNTY_folder = os.path.join(chart_folder, 'COUNTY')
if not os.path.exists(chart_COUNTY_folder):
    os.makedirs(chart_COUNTY_folder)
else:
    shutil.rmtree(chart_COUNTY_folder)
    os.makedirs(chart_COUNTY_folder)


new_df = new_se_county_dissolve_df.copy()
old_df = old_se_county_dissolve_df.copy()
id_col = 'CO_NAME'

categories = ('HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI')
cols = [col for col in new_df.columns if col.startswith(categories)]
new_new_cols = ['NEW_' + col for col in cols]
old_new_cols = ['OLD_' + col for col in cols]

new_rename_dict = dict(zip(cols, new_new_cols))
old_rename_dict = dict(zip(cols, old_new_cols))
new_df.rename(new_rename_dict, axis=1, inplace=True)
old_df.rename(old_rename_dict, axis=1, inplace=True)

df = new_df.merge(old_df, on=id_col, how='left')

for index, row in df.iterrows():
        f = open(os.path.join(chart_COUNTY_folder, "{}_{}.json".format(id_col,row[id_col])), "a")
        f.write("[\n")
        
        for cat in categories:  
            for year in range(2019,2051):
                
                new_col = '_'.join(['NEW', cat, str(year)])
                old_col = '_'.join(['OLD', cat, str(year)])

                f.write("\t{\n")
                f.write('''\t\t"C":"{}",\n'''.format(cat))
                f.write('''\t\t"Y":{},\n'''.format(year))
                
                f.write('''\t\t"N":{},\n'''.format(row[new_col]))
                f.write('''\t\t"O":{}\n'''.format(row[old_col]))
        
                if cat == "HJI" and year == 2050:
                    f.write("\t}\n")
                else:
                    f.write("\t},\n")

        f.write("]\n")
        f.close()

In [ ]:
#==============================
# add files to zipped folder
#==============================

d = date.today().strftime("%Y%m%d")

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))

with zipfile.ZipFile(os.path.join('.\\Outputs','Files_For_Web_App_{}.zip'.format(d)),'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir(chart_folder, zipf)
    zipdir(map_folder, zipf)

In [ ]:
# category_folder = os.path.join(chart_folder, c)
# if not os.path.exists(category_folder):
#     os.makedirs(category_folder)

# m = merged.copy()
# del m['SHAPE']
# m.to_json('file.json', orient='records', lines=True)
  

In [ ]:
# import arcgis.features
# from arcgis.features import GeoSeriesAccessor
# gsa = arcgis.features.GeoSeriesAccessor(new_temp_df['SHAPE'])
# new_temp_df['the_area'] = gsa.area
# new_temp_df

In [ ]:
# a = m.iloc[0]
# cols = [col for col in  m.columns if col.split('_')[0]=='N']
# f = open("CITYAREA_{}.json".format(a['CityArea']), "a")


# f.write("[\n")


# for col in cols:
#     f.write("\t{\n")
#     f.write('''\t\t"C":"{}",\n'''.format(c))
#     f.write('''\t\t"Y":{},\n'''.format(col.split('_')[1]))
#     f.write('''\t\t"V":{}\n'''.format(a[col]))
    
#     if col.split('_')[1] != "2050":
#         f.write("\t},\n")
#     else:
#         f.write("\t}\n")

# f.write("]\n")



# f.close()